In [5]:
import random
import math
import copy
from collections import deque

# --- DATASET ---
N_JOBS = 4
M_MACHINES = 2
L_SPEEDS = 3

# p_jm: processing time of job j on machine m (at normal speed r_l=1)
# p_values[machine_idx][job_idx]
p_values_str = {
    "p1j": "13 40 34 39", # M1
    "p2j": "23 89 95 5"   # M2
}
p_values = [
    [int(x) for x in p_values_str["p1j"].split()],
    [int(x) for x in p_values_str["p2j"].split()]
]

# r_l: processing speed factor
# r_values[speed_level_idx] (0:fast, 1:normal, 2:slow)
r_values_str = "1.2 1 0.8"
r_values = [float(x) for x in r_values_str.split()]

# beta_l: conversion factor associated with processing speed l
# beta_values[speed_level_idx]
beta_values_str = "1.2 1 0.64"
beta_values = [float(x) for x in beta_values_str.split()]

# gamma_m: conversion factor for idle time on machine m
# gamma_values[machine_idx]
gamma_values_str = "0.05 0.05"
gamma_values = [float(x) for x in gamma_values_str.split()]

# d_jkm: sequence-dependent set-up time from job j to job k on machine m
# S1jk_values[from_job_idx][to_job_idx]
s1jk_str = [
    "22 20 4 2",
    "4 11 20 8",
    "10 5 16 16",
    "4 25 1 22"
]
d_jkm_m1 = [[int(x) for x in row.split()] for row in s1jk_str]

s2jk_str = [
    "19 21 9 20",
    "22 23 4 19",
    "22 18 17 4",
    "4 2 10 18"
]
d_jkm_m2 = [[int(x) for x in row.split()] for row in s2jk_str]

# Assumed parameters
s_j_values = [0.0] * N_JOBS  # s_j: setup offset for job j on M2. Assume 0.
rho_m_values = [1.0] * M_MACHINES # rho_m: power consumption rate of machine m. Assume 1.0.



In [7]:
rho_m_values

[1.0, 1.0]

In [8]:
# --- OBJECTIVE FUNCTION CALCULATION ---
def calculate_objectives(sequence, speeds_assignment):
    """
    Calculates all four objectives based on a given job sequence and speed assignments.
    sequence: list of job indices (0 to N_JOBS-1)
    speeds_assignment: list of lists, speeds_assignment[job_idx][machine_idx] = speed_level (0,1,2)
    """
    job_completion_times_m1 = [0.0] * N_JOBS
    job_completion_times_m2 = [0.0] * N_JOBS
    actual_proc_times = [[0.0] * M_MACHINES for _ in range(N_JOBS)] # actual_proc_times[job_idx][machine_idx]

    # Calculate actual processing times based on speeds
    for job_idx_orig in range(N_JOBS):
        for machine_idx in range(M_MACHINES):
            speed_level = speeds_assignment[job_idx_orig][machine_idx]
            base_proc_time = p_values[machine_idx][job_idx_orig]
            actual_proc_times[job_idx_orig][machine_idx] = base_proc_time / r_values[speed_level]

    # Calculate completion times
    # Machine 1
    last_completion_m1 = 0.0
    for i, current_job_idx in enumerate(sequence):
        setup_time_m1 = 0
        if i == 0: # First job in sequence
            # d_jj1: initial setup for job current_job_idx on M1
            setup_time_m1 = d_jkm_m1[current_job_idx][current_job_idx]
            start_time_m1 = setup_time_m1
        else:
            prev_job_idx = sequence[i-1]
            # d_prev,current,1
            setup_time_m1 = d_jkm_m1[prev_job_idx][current_job_idx]
            start_time_m1 = job_completion_times_m1[prev_job_idx] + setup_time_m1
        
        job_completion_times_m1[current_job_idx] = start_time_m1 + actual_proc_times[current_job_idx][0]
        last_completion_m1 = job_completion_times_m1[current_job_idx]

    # Machine 2
    last_completion_m2 = 0.0
    for i, current_job_idx in enumerate(sequence):
        setup_time_m2 = 0
        
        # Earliest start on M2 due to M1 completion for current job
        earliest_start_m2_after_m1 = job_completion_times_m1[current_job_idx] + s_j_values[current_job_idx]
        
        earliest_start_m2_after_prev_job_m2 = 0
        if i == 0: # First job in sequence
            # d_jj2: initial setup for job current_job_idx on M2
            setup_time_m2 = d_jkm_m2[current_job_idx][current_job_idx]
            earliest_start_m2_after_prev_job_m2 = setup_time_m2
        else:
            prev_job_idx = sequence[i-1]
            # d_prev,current,2
            setup_time_m2 = d_jkm_m2[prev_job_idx][current_job_idx]
            earliest_start_m2_after_prev_job_m2 = job_completion_times_m2[prev_job_idx] + setup_time_m2
            
        start_time_m2 = max(earliest_start_m2_after_m1, earliest_start_m2_after_prev_job_m2)
        job_completion_times_m2[current_job_idx] = start_time_m2 + actual_proc_times[current_job_idx][1]
        last_completion_m2 = job_completion_times_m2[current_job_idx]

    # Objective 3: c_sup (Makespan)
    # c_sup is the completion time of the last job processed on the last machine.
    # This depends on which job is *physically* last on M2.
    # In a flow shop without job passing, the makespan is c_j2 of the job that is last in sequence.
    c_sup = job_completion_times_m2[sequence[-1]]


    # Objective 1: q (sum of job durations)
    q_val = 0.0
    for job_idx_orig in range(N_JOBS):
        for machine_idx in range(M_MACHINES):
            q_val += actual_proc_times[job_idx_orig][machine_idx]

    # Total processing time on each machine
    total_processing_on_m = [0.0] * M_MACHINES
    for job_idx_orig in range(N_JOBS):
        total_processing_on_m[0] += actual_proc_times[job_idx_orig][0]
        total_processing_on_m[1] += actual_proc_times[job_idx_orig][1]
        
    # t_m: total idle time on machine m
    # This definition: c_sup - sum(actual proc times on machine m)
    # This can be tricky if a machine finishes early and waits.
    # The paper's formula t_m = c_sup - sum(P_jm / r_l * u_jml) is direct.
    t_m_values = [0.0] * M_MACHINES
    t_m_values[0] = c_sup - total_processing_on_m[0]
    t_m_values[1] = c_sup - total_processing_on_m[1]
    
    # Ensure non-negative idle times (could happen if machine 1 finishes much earlier than c_sup)
    # However, the problem definition for t_m seems to imply system-wide idle up to c_sup.
    # If t_m is strictly positive machine idle time (gaps), calculation is more complex.
    # Let's stick to the provided formula for t_m.
    # A machine is idle from 0 until its first job starts, between jobs, and after its last job until C_sup.
    
    # Objective 4: sum of t_m
    sum_t_m = sum(t_m_values)
    if sum_t_m < -1e-6 : # account for small float errors
        # This check is important. If t_m can be negative, obj2 would be wrong.
        # print(f"Warning: Negative sum_t_m: {sum_t_m}. c_sup: {c_sup}, proc_m1: {total_processing_on_m[0]}, proc_m2: {total_processing_on_m[1]}")
        # Forcing non-negative as per t_m >= 0 domain.
        # This usually means the C_sup isn't correctly reflecting M1's actual finish if M1 is bottleneck
        # but the formula given IS c_sup - sum(proc).
        # Let's enforce t_m >=0 as per constraints. This usually means t_m = max(0, c_sup - total_proc_on_m)
        t_m_values[0] = max(0, c_sup - total_processing_on_m[0])
        t_m_values[1] = max(0, c_sup - total_processing_on_m[1])
        sum_t_m = sum(t_m_values)


    # Objective 2: e (total energy consumption)
    energy_processing = 0.0
    for job_idx_orig in range(N_JOBS):
        for machine_idx in range(M_MACHINES):
            speed_level = speeds_assignment[job_idx_orig][machine_idx]
            base_proc_time = p_values[machine_idx][job_idx_orig]
            # (p_jm / (60*r_l)) * beta_l * rho_m
            energy_processing += (base_proc_time / (60.0 * r_values[speed_level])) * \
                                 beta_values[speed_level] * rho_m_values[machine_idx]
    
    energy_idle = 0.0
    for machine_idx in range(M_MACHINES):
        # (gamma_m * rho_m / 60) * t_m
        energy_idle += (gamma_values[machine_idx] * rho_m_values[machine_idx] / 60.0) * t_m_values[machine_idx]
        
    e_val = energy_processing + energy_idle

    return {
        "obj1_q": q_val,
        "obj2_e": e_val,
        "obj3_c_sup": c_sup,
        "obj4_sum_t_m": sum_t_m,
        "details": { # For debugging or more info
            "job_comp_m1": job_completion_times_m1,
            "job_comp_m2": job_completion_times_m2,
            "actual_proc_times": actual_proc_times,
            "t_m_values": t_m_values
        }
    }



In [9]:
# --- TABU SEARCH COMPONENTS ---
def get_initial_solution():
    # Random sequence
    sequence = list(range(N_JOBS))
    random.shuffle(sequence)
    
    # Random speeds (or fixed, e.g., all normal speed = index 1)
    speeds = [[random.randint(0, L_SPEEDS - 1) for _ in range(M_MACHINES)] for _ in range(N_JOBS)]
    # Or fixed speeds:
    # speeds = [[1 for _ in range(M_MACHINES)] for _ in range(N_JOBS)] # All normal
    return {"sequence": sequence, "speeds": speeds}

def get_neighbors(current_solution):
    neighbors = []
    current_seq = current_solution["sequence"]
    current_speeds = current_solution["speeds"]

    # 1. Job swap neighbors
    for i in range(N_JOBS):
        for j in range(i + 1, N_JOBS):
            new_seq = current_seq[:]
            new_seq[i], new_seq[j] = new_seq[j], new_seq[i]
            # The move is defined by the *jobs* swapped, not their positions, to make tabu list more effective
            # Original jobs at pos i and j
            job1_idx = current_seq[i]
            job2_idx = current_seq[j]
            move_info = ("swap", tuple(sorted((job1_idx, job2_idx))))
            neighbors.append({"sequence": new_seq, "speeds": copy.deepcopy(current_speeds), "move": move_info})

    # 2. Speed change neighbors
    for job_idx in range(N_JOBS):
        for machine_idx in range(M_MACHINES):
            original_speed_level = current_speeds[job_idx][machine_idx]
            for speed_level in range(L_SPEEDS):
                if speed_level != original_speed_level:
                    new_speeds = copy.deepcopy(current_speeds)
                    new_speeds[job_idx][machine_idx] = speed_level
                    # Move info: (job_idx, machine_idx, new_speed_level_it_was_changed_TO)
                    # Tabu list should store (job_idx, machine_idx, original_speed_level)
                    # to prevent changing it back to original_speed_level
                    move_info = ("speed_change", (job_idx, machine_idx, original_speed_level))
                    neighbors.append({"sequence": copy.deepcopy(current_seq), "speeds": new_speeds, "move": move_info})
    return neighbors

# --- TABU SEARCH ALGORITHM ---
def tabu_search(max_iterations, tabu_tenure, num_runs=10):
    overall_best_solution = None
    overall_best_objectives = {
        "obj1_q": float('inf'),
        "obj2_e": float('inf'),
        "obj3_c_sup": float('inf'),
        "obj4_sum_t_m": float('inf')
    }
    
    print(f"Running Tabu Search for {num_runs} independent runs...")

    for run in range(num_runs):
        print(f"\n--- Run {run + 1}/{num_runs} ---")
        current_solution = get_initial_solution()
        current_objectives = calculate_objectives(current_solution["sequence"], current_solution["speeds"])
        
        best_solution_this_run = copy.deepcopy(current_solution)
        best_objectives_this_run = copy.deepcopy(current_objectives)

        tabu_list = deque(maxlen=tabu_tenure) # Stores moves that are tabu

        for iteration in range(max_iterations):
            neighbors = get_neighbors(current_solution)
            best_neighbor = None
            best_neighbor_objectives = None
            
            # Primary objective for guiding search: makespan (obj3_c_sup)
            current_best_neighbor_c_sup = float('inf')

            for neighbor_solution_struct in neighbors:
                # neighbor_solution_struct contains "sequence", "speeds", "move"
                move = neighbor_solution_struct["move"]

                # Aspiration: Allow tabu move if it's better than overall best found so far in this run
                is_tabu = move in tabu_list
                
                neighbor_objectives = calculate_objectives(neighbor_solution_struct["sequence"], neighbor_solution_struct["speeds"])
                
                aspirated = False
                if is_tabu:
                    if neighbor_objectives["obj3_c_sup"] < best_objectives_this_run["obj3_c_sup"]:
                        aspirated = True
                    else:
                        continue # Skip tabu non-aspirated move

                if neighbor_objectives["obj3_c_sup"] < current_best_neighbor_c_sup:
                    current_best_neighbor_c_sup = neighbor_objectives["obj3_c_sup"]
                    best_neighbor = neighbor_solution_struct
                    best_neighbor_objectives = neighbor_objectives
                elif neighbor_objectives["obj3_c_sup"] == current_best_neighbor_c_sup:
                    # Tie-breaking (e.g., using another objective, or random)
                    # For now, just take the first one that's equally good or better
                    if best_neighbor is None : # First valid neighbor
                        best_neighbor = neighbor_solution_struct
                        best_neighbor_objectives = neighbor_objectives


            if best_neighbor is None:
                # print(f"Iteration {iteration + 1}: No non-tabu improving neighbor found.")
                # This can happen if all moves are tabu and none meet aspiration
                # Or if neighborhood is empty (should not happen with current get_neighbors)
                # We might need a diversification strategy here, or just continue hoping tabu expires
                # For simplicity, we'll just break or let it run if tabu list expires items
                if not neighbors: # Should not happen
                    print("No neighbors generated.")
                    break
                # If all neighbors are tabu and non-aspirating, pick a random non-tabu or oldest tabu if desperate
                # For now, let's assume we can always find *some* move, even if it's worse.
                # Tabu search allows non-improving moves.
                # If all neighbors are tabu and not aspirated, we pick the LEAST WORSE tabu one (or random one if all same quality)
                # The current logic only picks if non-tabu OR tabu & aspirated.
                # If ALL neighbors are tabu and NONE meet aspiration, best_neighbor is None.
                # We need to pick the best one AMONG THEM (the least bad, even if tabu and not asp.)
                
                # Fallback: if no "good" neighbor, pick the best of all generated neighbors (even if tabu and not aspirating)
                # to ensure the search continues.
                # This part makes sure we always make a move.
                if not best_neighbor and neighbors:
                    temp_best_obj_val = float('inf')
                    temp_best_neigh = None
                    temp_best_neigh_obj = None
                    for neigh_s in neighbors:
                        neigh_o = calculate_objectives(neigh_s["sequence"], neigh_s["speeds"])
                        if neigh_o["obj3_c_sup"] < temp_best_obj_val:
                            temp_best_obj_val = neigh_o["obj3_c_sup"]
                            temp_best_neigh = neigh_s
                            temp_best_neigh_obj = neigh_o
                    best_neighbor = temp_best_neigh
                    best_neighbor_objectives = temp_best_neigh_obj

            if best_neighbor is None: # Should absolutely not happen now
                 print("Critical error: No neighbor selected.")
                 break


            current_solution = {"sequence": best_neighbor["sequence"], "speeds": best_neighbor["speeds"]}
            current_objectives = best_neighbor_objectives
            
            # Add the performed move to tabu list
            # The 'move' stored in best_neighbor is what needs to be made tabu
            tabu_list.append(best_neighbor["move"])

            # Update best solution for this run
            if current_objectives["obj3_c_sup"] < best_objectives_this_run["obj3_c_sup"]:
                best_solution_this_run = copy.deepcopy(current_solution)
                best_objectives_this_run = copy.deepcopy(current_objectives)
                # print(f"Iter {iteration + 1}: New best in run -> C_sup: {best_objectives_this_run['obj3_c_sup']:.2f}")
            
            if (iteration + 1) % 50 == 0 :
                 print(f"Run {run+1}, Iter {iteration + 1}: Current C_sup: {current_objectives['obj3_c_sup']:.2f}, Best in run C_sup: {best_objectives_this_run['obj3_c_sup']:.2f}")


        print(f"Run {run + 1} finished. Best C_sup for this run: {best_objectives_this_run['obj3_c_sup']:.2f}")
        print(f"  Objectives: Q={best_objectives_this_run['obj1_q']:.2f}, E={best_objectives_this_run['obj2_e']:.4f}, Sum_t_m={best_objectives_this_run['obj4_sum_t_m']:.2f}")
        print(f"  Sequence: {best_solution_this_run['sequence']}")
        # print(f"  Speeds: {best_solution_this_run['speeds']}")


        # Update overall best solution if this run was better
        if best_objectives_this_run["obj3_c_sup"] < overall_best_objectives["obj3_c_sup"]:
            overall_best_objectives = copy.deepcopy(best_objectives_this_run)
            overall_best_solution = copy.deepcopy(best_solution_this_run)
        elif best_objectives_this_run["obj3_c_sup"] == overall_best_objectives["obj3_c_sup"]:
            # Example tie-breaking: choose one with lower energy if makespan is same
            if best_objectives_this_run["obj2_e"] < overall_best_objectives["obj2_e"]:
                overall_best_objectives = copy.deepcopy(best_objectives_this_run)
                overall_best_solution = copy.deepcopy(best_solution_this_run)
                
    print("\n--- Overall Best Solution Found Across All Runs ---")
    if overall_best_solution:
        print(f"Best Sequence: {overall_best_solution['sequence']}")
        print(f"Best Speeds (job_idx -> [m1_speed_lvl, m2_speed_lvl]):")
        for j_idx, s_assign in enumerate(overall_best_solution['speeds']):
            print(f"  Job {j_idx}: M1_speed_idx={s_assign[0]}, M2_speed_idx={s_assign[1]}")
        
        print("\nObjectives for Overall Best Solution:")
        print(f"  Obj1 (q - sum of durations):         {overall_best_objectives['obj1_q']:.2f}")
        print(f"  Obj2 (e - total energy):           {overall_best_objectives['obj2_e']:.4f}")
        print(f"  Obj3 (c_sup - makespan):           {overall_best_objectives['obj3_c_sup']:.2f}")
        print(f"  Obj4 (sum_t_m - total idle time):  {overall_best_objectives['obj4_sum_t_m']:.2f}")
        # print(f"  Details: {overall_best_objectives['details']}") # Uncomment for more debug
    else:
        print("No solution found (should not happen).")

    return overall_best_solution, overall_best_objectives

# --- Parameters for Tabu Search ---
MAX_ITERATIONS = 200  # Iterations per run
TABU_TENURE = 7    # Size of tabu list (number of iterations a move is tabu)
NUM_INDEPENDENT_RUNS = 10 

# --- Run the Tabu Search ---
if __name__ == "__main__":
    # Sanity check calculation with a known solution
    # test_seq = [0, 1, 2, 3] # Job 0, then 1, then 2, then 3
    # test_speeds = [[1,1], [1,1], [1,1], [1,1]] # All normal speed
    # test_obj = calculate_objectives(test_seq, test_speeds)
    # print("Sanity Check Objectives (Seq 0-1-2-3, All Normal Speed):")
    # for k, v in test_obj.items():
    #     if k != "details":
    #         print(f"  {k}: {v}")
    #     else:
    #         print(f"  {k}: ...") # skip printing full details dict for brevity
    # print("-" * 20)

    best_sol, best_obj = tabu_search(MAX_ITERATIONS, TABU_TENURE, NUM_INDEPENDENT_RUNS)
    
    # Regarding "find a target value":
    # The Tabu search is designed to find a "good" solution by minimizing c_sup.
    # If you have a specific target value for c_sup (or any other objective),
    # you would typically run the search and see if the found best_obj['obj3_c_sup']
    # meets or beats that target. The search itself doesn't take a "target" as input
    # to stop early, but you could modify it to do so.

Running Tabu Search for 10 independent runs...

--- Run 1/10 ---
Run 1, Iter 50: Current C_sup: 251.67, Best in run C_sup: 251.67
Run 1, Iter 100: Current C_sup: 251.67, Best in run C_sup: 251.67
Run 1, Iter 150: Current C_sup: 252.50, Best in run C_sup: 251.67
Run 1, Iter 200: Current C_sup: 251.67, Best in run C_sup: 251.67
Run 1 finished. Best C_sup for this run: 251.67
  Objectives: Q=295.83, E=5.6929, Sum_t_m=207.50
  Sequence: [0, 1, 2, 3]

--- Run 2/10 ---
Run 2, Iter 50: Current C_sup: 233.00, Best in run C_sup: 233.00
Run 2, Iter 100: Current C_sup: 233.83, Best in run C_sup: 233.00
Run 2, Iter 150: Current C_sup: 233.00, Best in run C_sup: 233.00
Run 2, Iter 200: Current C_sup: 233.00, Best in run C_sup: 233.00
Run 2 finished. Best C_sup for this run: 233.00
  Objectives: Q=281.67, E=5.7869, Sum_t_m=184.33
  Sequence: [1, 2, 3, 0]

--- Run 3/10 ---
Run 3, Iter 50: Current C_sup: 250.00, Best in run C_sup: 250.00
Run 3, Iter 100: Current C_sup: 250.00, Best in run C_sup: 250.0

In [4]:
import random
import math
import copy
from collections import deque
import numpy as np # Import NumPy

# --- DATASET (Using NumPy arrays where appropriate) ---
N_JOBS = 4
M_MACHINES = 2
L_SPEEDS = 3

# p_jm: processing time of job j on machine m (at normal speed r_l=1)
# p_jm[machine_idx, job_idx]
p_jm = np.array([
    [13, 40, 34, 39],  # M1 (machine 0)
    [23, 89, 95, 5]   # M2 (machine 1)
], dtype=float)

# r_l: processing speed factor
# r_l[speed_level_idx] (0:fast, 1:normal, 2:slow)
r_l = np.array([1.2, 1.0, 0.8], dtype=float)

# beta_l: conversion factor associated with processing speed l
# beta_l[speed_level_idx]
beta_l = np.array([1.2, 1.0, 0.64], dtype=float)

# gamma_m: conversion factor for idle time on machine m
# gamma_m[machine_idx]
gamma_m = np.array([0.05, 0.05], dtype=float)

# d_jkm: sequence-dependent set-up time from job j to job k on machine m
# d_jkm[machine_idx, from_job_idx, to_job_idx]
d_jkm = np.array([
    [ # Machine 0 (M1)
        [22, 20, 4,  2],  # From job 0 to job 0, 1, 2, 3
        [4,  11, 20, 8],  # From job 1 to job 0, 1, 2, 3
        [10, 5,  16, 16], # From job 2 to job 0, 1, 2, 3
        [4,  25, 1,  22]  # From job 3 to job 0, 1, 2, 3
    ],
    [ # Machine 1 (M2)
        [19, 21, 9,  20], # From job 0 to job 0, 1, 2, 3
        [22, 23, 4,  19], # From job 1 to job 0, 1, 2, 3
        [22, 18, 17, 4],  # From job 2 to job 0, 1, 2, 3
        [4,  2,  10, 18]  # From job 3 to job 0, 1, 2, 3
    ]
], dtype=float)

# Assumed parameters
s_j = np.array([0.0] * N_JOBS, dtype=float) # s_j: setup offset for job j on M2. Assume 0.
rho_m = np.array([1.0] * M_MACHINES, dtype=float) # rho_m: power consumption rate of machine m. Assume 1.0.


# --- OBJECTIVE FUNCTION CALCULATION (using NumPy arrays) ---
def calculate_objectives_np(sequence, speeds_assignment_np):
    """
    Calculates all four objectives based on a given job sequence and speed assignments.
    sequence: list of job indices (0 to N_JOBS-1)
    speeds_assignment_np: NumPy array, speeds_assignment_np[job_idx, machine_idx] = speed_level (0,1,2)
    """
    job_completion_times_m1 = np.zeros(N_JOBS, dtype=float)
    job_completion_times_m2 = np.zeros(N_JOBS, dtype=float)
    
    # actual_proc_times_np[job_idx_orig, machine_idx]
    actual_proc_times_np = np.zeros((N_JOBS, M_MACHINES), dtype=float)

    # Calculate actual processing times based on speeds
    for job_idx_orig in range(N_JOBS):
        for machine_idx in range(M_MACHINES):
            speed_level = int(speeds_assignment_np[job_idx_orig, machine_idx])
            base_proc_time = p_jm[machine_idx, job_idx_orig]
            actual_proc_times_np[job_idx_orig, machine_idx] = base_proc_time / r_l[speed_level]

    # --- Calculate completion times ---
    # Machine 1 (machine_idx = 0)
    for i, current_job_idx in enumerate(sequence):
        setup_time_m1 = 0.0
        if i == 0: # First job in sequence
            # d_jj1: initial setup for job current_job_idx on M1 (d_jkm[0, current_job_idx, current_job_idx])
            setup_time_m1 = d_jkm[0, current_job_idx, current_job_idx]
            start_time_m1 = setup_time_m1
        else:
            prev_job_idx = sequence[i-1]
            # d_prev,current,1
            setup_time_m1 = d_jkm[0, prev_job_idx, current_job_idx]
            # Completion time of previous job on M1 is stored using original job index
            start_time_m1 = job_completion_times_m1[prev_job_idx] + setup_time_m1
        
        # Store completion time using original job index
        job_completion_times_m1[current_job_idx] = start_time_m1 + actual_proc_times_np[current_job_idx, 0]

    # Machine 2 (machine_idx = 1)
    for i, current_job_idx in enumerate(sequence):
        setup_time_m2 = 0.0
        
        # Earliest start on M2 due to M1 completion for current job
        earliest_start_m2_after_m1 = job_completion_times_m1[current_job_idx] + s_j[current_job_idx]
        
        earliest_start_m2_after_prev_job_m2 = 0.0
        if i == 0: # First job in sequence
            # d_jj2: initial setup for job current_job_idx on M2
            setup_time_m2 = d_jkm[1, current_job_idx, current_job_idx]
            earliest_start_m2_after_prev_job_m2 = setup_time_m2
        else:
            prev_job_idx = sequence[i-1]
            # d_prev,current,2
            setup_time_m2 = d_jkm[1, prev_job_idx, current_job_idx]
            earliest_start_m2_after_prev_job_m2 = job_completion_times_m2[prev_job_idx] + setup_time_m2
            
        start_time_m2 = max(earliest_start_m2_after_m1, earliest_start_m2_after_prev_job_m2)
        job_completion_times_m2[current_job_idx] = start_time_m2 + actual_proc_times_np[current_job_idx, 1]

    # Objective 3: c_sup (Makespan)
    # c_sup is the completion time of the job that is last *in sequence* on the last machine.
    c_sup = job_completion_times_m2[sequence[-1]]

    # Objective 1: q (sum of job durations)
    q_val = np.sum(actual_proc_times_np)

    # Total processing time on each machine
    total_processing_on_m = np.sum(actual_proc_times_np, axis=0) # Sum over jobs for each machine
        
    # t_m: total idle time on machine m
    t_m_values = np.zeros(M_MACHINES, dtype=float)
    t_m_values[0] = max(0, c_sup - total_processing_on_m[0])
    t_m_values[1] = max(0, c_sup - total_processing_on_m[1])
    
    # Objective 4: sum of t_m
    sum_t_m = np.sum(t_m_values)

    # Objective 2: e (total energy consumption)
    energy_processing = 0.0
    for job_idx_orig in range(N_JOBS):
        for machine_idx in range(M_MACHINES):
            speed_level = int(speeds_assignment_np[job_idx_orig, machine_idx])
            base_proc_time = p_jm[machine_idx, job_idx_orig]
            energy_processing += (base_proc_time / (60.0 * r_l[speed_level])) * \
                                 beta_l[speed_level] * rho_m[machine_idx]
    
    energy_idle = 0.0
    for machine_idx in range(M_MACHINES):
        energy_idle += (gamma_m[machine_idx] * rho_m[machine_idx] / 60.0) * t_m_values[machine_idx]
        
    e_val = energy_processing + energy_idle

    return {
        "obj1_q": q_val,
        "obj2_e": e_val,
        "obj3_c_sup": c_sup,
        "obj4_sum_t_m": sum_t_m,
        "details": { # For debugging or more info
            "job_comp_m1": job_completion_times_m1.tolist(), # Convert to list for printing if needed
            "job_comp_m2": job_completion_times_m2.tolist(),
            "actual_proc_times": actual_proc_times_np.tolist(),
            "t_m_values": t_m_values.tolist()
        }
    }

# --- TABU SEARCH COMPONENTS (largely same, but ensure speeds are NumPy arrays) ---
def get_initial_solution_np():
    sequence = list(range(N_JOBS))
    random.shuffle(sequence)
    
    # speeds_np[job_idx, machine_idx]
    speeds_np = np.random.randint(0, L_SPEEDS, size=(N_JOBS, M_MACHINES))
    # Or fixed speeds:
    # speeds_np = np.full((N_JOBS, M_MACHINES), 1, dtype=int) # All normal speed (index 1)
    return {"sequence": sequence, "speeds_np": speeds_np}

def get_neighbors_np(current_solution):
    neighbors = []
    current_seq = current_solution["sequence"]
    current_speeds_np = current_solution["speeds_np"]

    # 1. Job swap neighbors
    for i in range(N_JOBS):
        for j in range(i + 1, N_JOBS):
            new_seq = current_seq[:]
            new_seq[i], new_seq[j] = new_seq[j], new_seq[i]
            job1_idx = current_seq[i]
            job2_idx = current_seq[j]
            move_info = ("swap", tuple(sorted((job1_idx, job2_idx))))
            # Make sure to pass a copy of the NumPy array
            neighbors.append({"sequence": new_seq, "speeds_np": current_speeds_np.copy(), "move": move_info})

    # 2. Speed change neighbors
    for job_idx in range(N_JOBS):
        for machine_idx in range(M_MACHINES):
            original_speed_level = int(current_speeds_np[job_idx, machine_idx])
            for speed_level in range(L_SPEEDS):
                if speed_level != original_speed_level:
                    new_speeds_np = current_speeds_np.copy()
                    new_speeds_np[job_idx, machine_idx] = speed_level
                    move_info = ("speed_change", (job_idx, machine_idx, original_speed_level))
                    neighbors.append({"sequence": current_seq[:], "speeds_np": new_speeds_np, "move": move_info})
    return neighbors

# --- TABU SEARCH ALGORITHM (Modified to use _np versions) ---
def tabu_search_np(max_iterations, tabu_tenure, num_runs=10, target_makespan_to_find=None):
    overall_best_solution_np = None
    overall_best_objectives_np = {
        "obj1_q": float('inf'), "obj2_e": float('inf'),
        "obj3_c_sup": float('inf'), "obj4_sum_t_m": float('inf')
    }
    
    print(f"Running Tabu Search with NumPy data for {num_runs} independent runs...")
    found_target_flag = False

    for run in range(num_runs):
        if found_target_flag and target_makespan_to_find is not None: # Optional: stop all runs if target met
            break
        print(f"\n--- Run {run + 1}/{num_runs} ---")
        current_solution_np = get_initial_solution_np()
        current_objectives_np = calculate_objectives_np(current_solution_np["sequence"], current_solution_np["speeds_np"])
        
        best_solution_this_run_np = copy.deepcopy(current_solution_np)
        best_objectives_this_run_np = copy.deepcopy(current_objectives_np)

        tabu_list = deque(maxlen=tabu_tenure)

        for iteration in range(max_iterations):
            neighbors = get_neighbors_np(current_solution_np)
            best_neighbor_np = None
            best_neighbor_objectives_np = None
            current_best_neighbor_c_sup = float('inf')

            for neighbor_solution_struct in neighbors:
                move = neighbor_solution_struct["move"]
                is_tabu = move in tabu_list
                
                neighbor_objectives = calculate_objectives_np(neighbor_solution_struct["sequence"], neighbor_solution_struct["speeds_np"])
                
                aspirated = False
                if is_tabu:
                    if neighbor_objectives["obj3_c_sup"] < best_objectives_this_run_np["obj3_c_sup"]:
                        aspirated = True
                    else:
                        continue

                if not is_tabu or aspirated:
                    if neighbor_objectives["obj3_c_sup"] < current_best_neighbor_c_sup:
                        current_best_neighbor_c_sup = neighbor_objectives["obj3_c_sup"]
                        best_neighbor_np = neighbor_solution_struct
                        best_neighbor_objectives_np = neighbor_objectives
                    elif neighbor_objectives["obj3_c_sup"] == current_best_neighbor_c_sup:
                         if best_neighbor_np is None : 
                            best_neighbor_np = neighbor_solution_struct
                            best_neighbor_objectives_np = neighbor_objectives

            if best_neighbor_np is None: # Fallback if all valid moves were non-improving or only tabu non-aspirated
                if neighbors: # Pick the best among all (even if worse or tabu non-aspirated)
                    temp_best_val = float('inf')
                    for n_struct in neighbors:
                        n_obj = calculate_objectives_np(n_struct["sequence"], n_struct["speeds_np"])
                        if n_obj["obj3_c_sup"] < temp_best_val:
                            temp_best_val = n_obj["obj3_c_sup"]
                            best_neighbor_np = n_struct
                            best_neighbor_objectives_np = n_obj
                else: # No neighbors generated, should not happen
                    print("No neighbors generated.")
                    break 
            
            if best_neighbor_np is None: # Still none? Critical issue.
                 print("Critical error: No neighbor selected despite fallback.")
                 break

            current_solution_np = {"sequence": best_neighbor_np["sequence"], "speeds_np": best_neighbor_np["speeds_np"]}
            current_objectives_np = best_neighbor_objectives_np
            tabu_list.append(best_neighbor_np["move"])

            if current_objectives_np["obj3_c_sup"] < best_objectives_this_run_np["obj3_c_sup"]:
                best_solution_this_run_np = copy.deepcopy(current_solution_np)
                best_objectives_this_run_np = copy.deepcopy(current_objectives_np)

            if (iteration + 1) % 50 == 0:
                print(f"Run {run+1}, Iter {iteration + 1}: Curr C_sup: {current_objectives_np['obj3_c_sup']:.2f}, Best in run C_sup: {best_objectives_this_run_np['obj3_c_sup']:.2f}")

            if target_makespan_to_find is not None and best_objectives_this_run_np["obj3_c_sup"] <= target_makespan_to_find:
                print(f"Target makespan {target_makespan_to_find} reached or beaten in Run {run+1}, Iter {iteration+1} with {best_objectives_this_run_np['obj3_c_sup']:.2f}!")
                found_target_flag = True
                break # Break from iterations for this run

        print(f"Run {run + 1} finished. Best C_sup for this run: {best_objectives_this_run_np['obj3_c_sup']:.2f}")
        print(f"  Objectives: Q={best_objectives_this_run_np['obj1_q']:.2f}, E={best_objectives_this_run_np['obj2_e']:.4f}, Sum_t_m={best_objectives_this_run_np['obj4_sum_t_m']:.2f}")
        print(f"  Sequence: {best_solution_this_run_np['sequence']}")

        if best_objectives_this_run_np["obj3_c_sup"] < overall_best_objectives_np["obj3_c_sup"]:
            overall_best_objectives_np = copy.deepcopy(best_objectives_this_run_np)
            overall_best_solution_np = copy.deepcopy(best_solution_this_run_np)
        elif best_objectives_this_run_np["obj3_c_sup"] == overall_best_objectives_np["obj3_c_sup"]:
            if best_objectives_this_run_np["obj2_e"] < overall_best_objectives_np["obj2_e"]:
                overall_best_objectives_np = copy.deepcopy(best_objectives_this_run_np)
                overall_best_solution_np = copy.deepcopy(best_solution_this_run_np)
                
    print("\n--- Overall Best Solution Found Across All Runs (NumPy version) ---")
    if overall_best_solution_np:
        print(f"Best Sequence: {overall_best_solution_np['sequence']}")
        print(f"Best Speeds (job_idx -> [m1_speed_lvl, m2_speed_lvl]):\n{overall_best_solution_np['speeds_np']}")
        
        print("\nObjectives for Overall Best Solution:")
        print(f"  Obj1 (q - sum of durations):         {overall_best_objectives_np['obj1_q']:.2f}")
        print(f"  Obj2 (e - total energy):           {overall_best_objectives_np['obj2_e']:.4f}")
        print(f"  Obj3 (c_sup - makespan):           {overall_best_objectives_np['obj3_c_sup']:.2f}  <--- Value to compare with your target")
        print(f"  Obj4 (sum_t_m - total idle time):  {overall_best_objectives_np['obj4_sum_t_m']:.2f}")
    else:
        print("No solution found.")
    
    if target_makespan_to_find is not None:
        if overall_best_objectives_np['obj3_c_sup'] <= target_makespan_to_find :
            print(f"\nSUCCESS: The best makespan found ({overall_best_objectives_np['obj3_c_sup']:.2f}) meets or beats the target ({target_makespan_to_find}).")
        else:
            print(f"\nINFO: The best makespan found ({overall_best_objectives_np['obj3_c_sup']:.2f}) did NOT meet the target ({target_makespan_to_find}).")


    return overall_best_solution_np, overall_best_objectives_np

# --- Parameters for Tabu Search ---
MAX_ITERATIONS = 200
TABU_TENURE = 7
NUM_INDEPENDENT_RUNS = 10
TARGET_MAKESPAN = None # Example: 200.0 or None if no specific target to stop early for

# --- Run the Tabu Search ---
if __name__ == "__main__":
    # Example: Test with a specific target makespan
    # You can set TARGET_MAKESPAN to a numeric value if you want the algorithm
    # to acknowledge if it finds a solution better than or equal to it.
    # If TARGET_MAKESPAN is None, it just tries to find the best possible.
    
    # Example: Set a target
    # TARGET_MAKESPAN = 180.0 
    # print(f"Attempting to find a makespan <= {TARGET_MAKESPAN}\n")

    best_sol_np, best_obj_np = tabu_search_np(
        MAX_ITERATIONS, 
        TABU_TENURE, 
        NUM_INDEPENDENT_RUNS,
        target_makespan_to_find=TARGET_MAKESPAN
    )

Running Tabu Search with NumPy data for 10 independent runs...

--- Run 1/10 ---
Run 1, Iter 50: Curr C_sup: 233.00, Best in run C_sup: 233.00
Run 1, Iter 100: Curr C_sup: 233.00, Best in run C_sup: 233.00
Run 1, Iter 150: Curr C_sup: 233.00, Best in run C_sup: 233.00
Run 1, Iter 200: Curr C_sup: 233.00, Best in run C_sup: 233.00
Run 1 finished. Best C_sup for this run: 233.00
  Objectives: Q=288.17, E=5.7815, Sum_t_m=177.83
  Sequence: [1, 2, 3, 0]

--- Run 2/10 ---
Run 2, Iter 50: Curr C_sup: 251.67, Best in run C_sup: 251.67
Run 2, Iter 100: Curr C_sup: 251.67, Best in run C_sup: 251.67
Run 2, Iter 150: Curr C_sup: 252.50, Best in run C_sup: 251.67
Run 2, Iter 200: Curr C_sup: 251.67, Best in run C_sup: 251.67
Run 2 finished. Best C_sup for this run: 251.67
  Objectives: Q=295.83, E=5.6929, Sum_t_m=207.50
  Sequence: [0, 1, 2, 3]

--- Run 3/10 ---
Run 3, Iter 50: Curr C_sup: 233.00, Best in run C_sup: 233.00
Run 3, Iter 100: Curr C_sup: 233.00, Best in run C_sup: 233.00
Run 3, Iter 